In [12]:
import os
import sys
from pathlib import Path
from loguru import logger

import monai_training
from monai_training.preprocess import DataSetProcesser
from monai_training import training
from mri_data import file_manager as fm

In [17]:
from reload_recursive import reload_recursive

reload_recursive(monai_training)
from monai_training import training

In [3]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [4]:
drive_root = fm.get_drive_root()
projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
dataroot = drive_root / "3Tpioneer_bids"

dataset_name = "single-vs-multi1"

In [5]:
subjects = [
    1010,
    1011,
    1019,
    1029,
    1033,
    1037,
    1038,
    1080,
    1085,
    1087,
    1089,
    1109,
    1119,
    1152,
    1163,
    1188,
    1191,
    1234,
    1259,
    1265,
    1272,
    1280,
    1293,
    1321,
    1355,
    1437,
    1498,
    1518,
    1540,
    1547,
    1548,
    2081,
    2083,
    2097,
    2126,
    2132,
    2144,
    2146,
    2164,
    2187]

In [6]:
def has_subject(scan, subjects: list[int]) -> bool:
    if int(scan.subid) in subjects:
        return True
    else:
        return False

In [7]:
dataset_proc = DataSetProcesser.new_dataset(dataroot, fm.scan_3Tpioneer_bids, filters=[fm.filter_first_ses])
dataset_proc.filter([has_subject], [(subjects,)])
dataset = dataset_proc.dataset
dataset.sort()

In [28]:
for i,scan in enumerate(dataset):
    dataset[i].label = None
    dataset[i].image = None

In [29]:
from monai_training import preprocess


# dataset = training.assign_conditions(dataset, 0.25)
preprocess.save_dataset(dataset, "dataset.json")

In [26]:
for scan in dataset:
    if scan.cond == "ts":
        print(scan.subid)

1010
1033
1119
1259
1437
1547
2081
2126
2146
2187


In [27]:
dataset[0]

Scan(subid='1010', sesid='20180208', _dataroot=PosixPath('/mnt/h/3Tpioneer_bids'), _root=PosixPath('/mnt/h/3Tpioneer_bids/sub-ms1010/ses-20180208'), image='flair.t1.nii.gz', label='choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz', cond='ts', id=20382010080)

In [22]:
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(["choroid_t1_flair", "pineal", "pituitary"], ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(["flair", "t1"])

2024-12-10 20:44:05.696 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
  0%|          | 0/40 [00:00<?, ?it/s]2024-12-10 20:44:05.710 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1010, sesid=20180208)
2024-12-10 20:44:06.282 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms1010/ses-20180208/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz
  2%|▎         | 1/40 [00:00<00:22,  1.74it/s]2024-12-10 20:44:06.285 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1011, sesid=20180911)
2024-12-10 20:44:06.301 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms1011/ses-20180911/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz
2024-12-10 20:44:06.303 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1019, sesid=20190608)
2024-12-10 20:44:06.316 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpion

In [24]:
dataset = dataset_proc.dataset
dataset[0]

Scan(subid='1010', sesid='20180208', _dataroot=PosixPath('/mnt/h/3Tpioneer_bids'), _root=PosixPath('/mnt/h/3Tpioneer_bids/sub-ms1010/ses-20180208'), image='flair.t1.nii.gz', label='choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz', cond='tr', id=20382010080)